In [ ]:
import geopandas as gpd

In [ ]:
schools = gpd.read_file("../../geopandas_101_DATA/Forrest/data/nyc/SchoolPoints_APS_2024_08_28 (1)/SchoolPoints_APS_2024_08_28.shp")
subways = gpd.read_file("../../geopandas_101_DATA/Forrest/data/nyc/nyc_subway_entrances/nyc_subway_entrances.shp")
bike_paths = gpd.read_file("../../geopandas_101_DATA/Forrest/data/nyc/New York City Bike Routes_20241223.geojson")
neighborhoods = gpd.read_file("https://raw.githubusercontent.com/HodgesWardElliott/custom-nyc-neighborhoods/refs/heads/master/custom-pedia-cities-nyc-Mar2018.geojson")
parks = gpd.read_file("../../geopandas_101_DATA/Forrest/data/nyc/Parks Properties_20241223.geojson")

We need to have the same CRS in all dataframes. We will set it to 3857. This CRM measures in meters.

In [ ]:
frames = [schools, subways, bike_paths, neighborhoods, parks]
for frame in frames:
    print(frame.crs)

In [ ]:
schools = schools.to_crs("EPSG:3857")
subways = subways.to_crs("EPSG:3857")
bike_paths = bike_paths.to_crs("EPSG:3857")
neighborhoods = neighborhoods.to_crs("EPSG:3857")
parks = parks.to_crs("EPSG:3857")

In [ ]:
frames = [schools, subways, bike_paths, neighborhoods, parks]
for frame in frames:
    print(frame.crs)

In [ ]:
crs = "EPSG:3857"

uws = neighborhoods[neighborhoods["neighborhood"]=="Upper West Side"]
uws = uws.to_crs(crs)
schools = schools.to_crs(crs)

print(uws.crs)
print(schools.crs)

gpd.sjoin(schools, uws, how="inner", predicate="within").buffer(100).explore()

In [ ]:
crs = "EPSG:3857"

uws = neighborhoods[neighborhoods["neighborhood"]=="Upper West Side"]
uws = uws.to_crs(crs)
bike_paths = bike_paths.to_crs(crs)

print(uws.crs)
print(bike_paths.crs)

gpd.sjoin(bike_paths, uws, how="inner", predicate="within").explore()

In [ ]:
print(uws.crs)
print(schools.crs)

In [ ]:
uws.buffer(100).explore()

In [ ]:
uws.buffer(100).explore()

In [ ]:
uws.bounds

In [ ]:
schools.crs

In [ ]:
for frame in frames:
    print(frame.geom_type.unique())

In [ ]:
import pandas as pd

In [ ]:
pd.options.display.float_format = '{:,.4f}'.format

## Area

Extract the area of each neighborhood from the ```geometry``` column. It is measureed in square meters.

In [ ]:
# 1. GeoSeries.area
neighborhoods["area"] = neighborhoods.geometry.area
neighborhoods[['neighborhood', 'area']].sort_values(['area'], ascending=False).head(20)

## Boundary

In [ ]:
# 2. GeoSeries.boundary
parks["boundary"] = parks.geometry.boundary
parks_mapped = parks['boundary'].to_crs('EPSG:4326')

In [ ]:
# Notice that the geometry of parks_mapped are not polygons anymore; 
# they are now lines (the polygon's boundary lines)
print(parks.geom_type.unique())
print(parks_mapped.geom_type.unique())

In [ ]:
parks_mapped.explore()
# parks.geometry.boundary.explore()

In [ ]:
parks.explore()

## Bounds

In [ ]:
# parks_mapped.geometry.bounds
parks_mapped.bounds

In [ ]:
parks_mapped.geometry.total_bounds

In [ ]:
parks.to_crs("EPSG:4326").geometry.total_bounds

In [ ]:
parks.geometry.total_bounds

In [ ]:
from shapely.geometry import box


# Create bounding boxes as geometries
bounds = parks_mapped.bounds
boxes = [box(xmin, ymin, xmax, ymax) for xmin, ymin, xmax, ymax in bounds.values]

# Turn bounding boxes into a GeoDataFrame
bounds_gdf = gpd.GeoDataFrame(geometry=boxes, crs=parks_mapped.crs)

# Plot original geometries with bounds overlaid
ax = parks_mapped.plot(edgecolor='black', facecolor='lightgray')
bounds_gdf.plot(ax=ax, edgecolor='red', facecolor='none', linewidth=1)


[EPSG 2263](https://epsg.io/?q=2263) is specific to New York state and Long Island. It measures distances in feet.

In [ ]:
paths_2263 = bike_paths.to_crs('EPSG:2263')

In [ ]:
paths_2263["length"] = paths_2263.geometry.length
paths_2263[['segmentid', 'length']].sort_values(by=['length'], ascending=False)

In [ ]:
neighborhoods[neighborhoods['neighborhood'] == 'Chelsea']

In [ ]:
bike_paths[
    bike_paths['segmentid'].astype(str) == '299972.0'
    ].to_crs(
        'EPSG:4326'
        ).explore()

In [ ]:
schools["nearest_subway_distance"] = schools.geometry.apply(
    lambda school: subways.distance(school).min()
)

In [ ]:
[subways.distance(g).min() for g in schools.geometry]

In [ ]:
schools[['Name', 'nearest_subway_distance']].sort_values(by=['nearest_subway_distance'], ascending=True)

In [ ]:
schools[schools['Name']=='P.S. 150'].explore()

In [ ]:
neighborhoods.explore()

In [ ]:
neighborhoods.centroid.explore()

In [ ]:
bike_paths.geometry.buffer(10)

In [ ]:
bike_paths.geometry

In [ ]:
bike_paths.head(100).geometry.explore()

In [ ]:
bike_paths.head(100).geometry.buffer(100).explore()

In [ ]:
parks["intersects_bike_path"] = parks.geometry.apply(
    lambda park: bike_paths.geometry.intersects(park).any()
)

In [ ]:
bike_paths.geometry.intersects(parks.geometry[0])

In [ ]:
parks[parks['intersects_bike_path'] == True].explore()

In [ ]:
parks.geometry.minimum_bounding_circle().explore()

In [ ]:
parks[parks['eapply']=="Central Park"].geometry.minimum_bounding_circle().explore()

In [ ]:
parks[parks['eapply']=="Central Park"].geometry.envelope.explore()

In [ ]:
neighborhoods[neighborhoods['neighborhood']=="Upper West Side"].geometry.explore()

In [ ]:
neighborhoods[neighborhoods['neighborhood']=="Upper West Side"].geometry.simplify(tolerance=0.001).explore()

In [ ]:
neighborhoods[neighborhoods['neighborhood']=="Upper West Side"].geometry.convex_hull.explore()

In [ ]:
neighborhoods[neighborhoods['neighborhood']=="Upper West Side"].geometry.envelope.explore()

In [ ]:
neighborhoods[neighborhoods['neighborhood']=="Upper West Side"].geometry.minimum_bounding_circle().explore()

In [ ]:
neighborhoods.geometry.convex_hull.explore()

In [ ]:
schools.geometry.envelope.explore()

In [ ]:
schools.geometry.envelope.explore()

In [ ]:
parks_neighborhoods_overlay = neighborhoods.overlay(parks, how="intersection")

In [ ]:
parks_neighborhoods_overlay

In [ ]:
print(parks['geometry'])
print(neighborhoods['geometry'])
print(parks_neighborhoods_overlay['geometry'])

In [ ]:
parks.boundary.explore()

In [ ]:
parks.geometry.boundary.explore()

In [ ]:
print(len(parks_neighborhoods_overlay))
print(len(neighborhoods))

In [ ]:
parks_neighborhoods_overlay.explore()

In [ ]:
neighborhoods_parks_overlay.explore()

In [ ]:
neighborhoods_parks_overlay = parks.overlay(neighborhoods, how="intersection")

In [ ]:
neighborhoods_parks_overlay

In [ ]:
subways["translated"] = subways.geometry.translate(xoff=1000, yoff=1000)
subways.translated.explore()

In [ ]:
parks.geometry.is_valid

In [ ]:
neighborhoods.geometry.exterior.explore()

In [ ]:
bike_paths.geometry.geom_type


In [ ]:
schools["x"] = schools.geometry.to_crs(epsg=4326).x
schools["y"] = schools.geometry.to_crs(epsg=4326).y

In [ ]:
schools.info()

In [ ]:
schools[['x', 'Longitude', 'y', 'Latitude']]

In [ ]:
neighborhoods["contains_parks"] = neighborhoods.geometry.apply(
    lambda neighborhood: parks.geometry.apply(
        lambda park: neighborhood.contains(park)
    ).any()
)

In [ ]:
neighborhoods[neighborhoods['contains_parks'] == False].explore()

In [ ]:
bike_paths["crosses_parks"] = bike_paths.geometry.apply(
    lambda path: path.crosses(parks.geometry).any()
)

In [ ]:
bike_paths[bike_paths['crosses_parks'] == True].explore()

In [ ]:
parks["disjoint_neighborhoods"] = parks.geometry.apply(
    lambda park: park.disjoint(neighborhoods.geometry).all()
)

In [ ]:
parks[parks['disjoint_neighborhoods'] == True].explore()

In [ ]:
def schoolsWithin500(school):
    return school.distance(subways.geometry).any() <= 500

In [ ]:
schools["within_500m_subway"] = schools.geometry.apply(
    schoolsWithin500
)


In [ ]:
subway_union = subways.geometry.union_all()
schools["within_500m_subway"] = schools.geometry.distance(subway_union) <= 500


In [ ]:
schools[schools["within_500m_subway"] == True].explore()

In [ ]:
parks["overlaps_neighborhoods"] = parks.geometry.apply(
    lambda park: park.overlaps(neighborhoods.geometry).any()
)


In [ ]:
parks[parks["overlaps_neighborhoods"] == True].explore()

In [ ]:
parks['geometry'].symmetric_difference(neighborhoods['geometry']).explore()